In [1]:
import io
import os
import json
import pandas as pd

# Imports the Google Cloud client library
from google.cloud import vision_v1p1beta1
from google.cloud.vision_v1p1beta1 import types

In [2]:
%env GOOGLE_APPLICATION_CREDENTIALS=/Users/chappers/Documents/DataScience/0_Method_DS/MethodDS-KaggleKat-8b57553f8dbf.json

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/chappers/Documents/DataScience/0_Method_DS/MethodDS-KaggleKat-8b57553f8dbf.json


###  Example Image being used for classification

In [3]:
# Show image being used
import ipywidgets as widgets
file = open("IMG_2934.jpg", "rb")
image = file.read()
widgets.Image(
    value=image,
    format='jpg',
    width=300,
    height=400,)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00H\x00H\x00\x00\xff\xe1\x08\x1aExif\x00\x00MM\x00…

### Start of Code to classify the image utilising the Google Vision API

In [4]:
# Instantiates a client
client = vision_v1p1beta1.ImageAnnotatorClient()

# The name of the image file to annotate
file_name = os.path.join(
    os.path.dirname('__file__'),
    'IMG_2934.jpg')

In [5]:
# Loads the image into memory
with io.open(file_name, 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

# Performs label detection on the image file
response = client.label_detection(image=image)
labels = response.label_annotations

print('Labels:')
for label in labels:
    # print(label.description)
    print(label)

Labels:
mid: "/m/0n5v01m"
description: "bag"
score: 0.8551287651062012
topicality: 0.8551287651062012

mid: "/m/01jwgf"
description: "product"
score: 0.7906314134597778
topicality: 0.7906314134597778

mid: "/m/080hkjn"
description: "handbag"
score: 0.7440301775932312
topicality: 0.7440301775932312

mid: "/m/04t7l"
description: "metal"
score: 0.6029984951019287
topicality: 0.6029984951019287

mid: "/m/03hzcs"
description: "beige"
score: 0.5875622034072876
topicality: 0.5875622034072876

mid: "/m/0j62f"
description: "rectangle"
score: 0.5806199312210083
topicality: 0.5806199312210083

mid: "/m/0hgrzkp"
description: "shoulder bag"
score: 0.5463387966156006
topicality: 0.5463387966156006



In [6]:
# create str_search to be used in the EBAY API call for KEYWORDS
str_search = str(labels[0].description)

### Code to utilise the Best Guess Label within Google Vision API

In [7]:
web_scores=[]
best_g=[]

# def annotate(path):
#    """Returns web annotations given the path to an image."""
#    client = vision.ImageAnnotatorClient()

#    if path.startswith('http') or path.startswith('gs:'):
#        image = types.Image()
#        image.source.image_uri = path

#    else:
#        with io.open(path, 'rb') as image_file:
#            content = image_file.read()

#        image = vision.types.Image(content=content)

#    web_detection = client.web_detection(image=image).web_detection

#    return web_detection

def detect_web(path):
    """Detects web annotations given an image."""
    client = vision_v1p1beta1.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision_v1p1beta1.types.Image(content=content)

    # how doe we get this 
    response = client.web_detection(image=image)
    annotations = response.web_detection

    for label in annotations.best_guess_labels:
        print('\nBest guess label: {}'.format(label.label))
        best_g.append(format(label.label))

# def report(annotations):
    """Prints detected features in the provided web annotations."""
    if annotations.pages_with_matching_images:
        print('\n{} Pages with matching images retrieved'.format(
            len(annotations.pages_with_matching_images)))

        for page in annotations.pages_with_matching_images:
            print('Url   : {}'.format(page.url))
            
#    if annotations.best_guess_labels:
#        for label in annotations.best_guess_labels:
#            print('\nBest guess label: {}'.format(label.label))

    if annotations.full_matching_images:
        print ('\n{} Full Matches found: '.format(
               len(annotations.full_matching_images)))

        for image in annotations.full_matching_images:
            print('Url  : {}'.format(image.url))

    if annotations.partial_matching_images:
        print ('\n{} Partial Matches found: '.format(
               len(annotations.partial_matching_images)))

        for image in annotations.partial_matching_images:
            print('Url  : {}'.format(image.url))

    if annotations.web_entities:
        print ('\n{} Web entities found: '.format(
            len(annotations.web_entities)))

        for entity in annotations.web_entities:
            print('Score      : {}'.format(entity.score))
            print('Description: {}'.format(entity.description))
            if (entity.score>0.90):
                web_scores.append(entity.description)

In [8]:
# report(annotate('IMG_2934.jpg'))
detect_web('IMG_2934.jpg')


Best guess label: michael kors 35f6gfel3l jet set travel vanilla/acorn large carryall tote handbag

2 Pages with matching images retrieved
Url   : https://www.pinterest.com/LuvClearance/sperry-top-sider-boat-shoes/
Url   : https://www.pinterest.co.uk/pin/573927546241896605/

1 Partial Matches found: 
Url  : https://i.pinimg.com/736x/13/3d/6b/133d6b524fe45d2ba2546ab6a9021f8c--vanilla-flip-flops.jpg

10 Web entities found: 
Score      : 1.2327721118927002
Description: Handbag
Score      : 0.7045000195503235
Description: Flip-flops
Score      : 0.6243000030517578
Description: Shoe
Score      : 0.558899998664856
Description: Designer
Score      : 0.5447999835014343
Description: Sandal
Score      : 0.5371999740600586
Description: 
Score      : 0.519599974155426
Description: Tote bag
Score      : 0.5080999732017517
Description: Polo shirt
Score      : 0.47699999809265137
Description: Bag
Score      : 0.4677000045776367
Description: Ballet flat


In [9]:
for rows in range(0,len(web_scores)):
    str_search=(str_search+" , "+ web_scores[rows])

### Creating keyword string to pass to EBAY API

In [10]:
# Create the Keyword String to then feed into the Ebay API, note the use of brackets and comma's, according to API
# documentation, this then results in items being returned with any of words seperated by comma's, ie. OR logic
keyword_string = best_g[0]
# kstring = keyword_string.split(" ")
keyword_string = str("(" + keyword_string + ")")
keyword_string = keyword_string.replace(" ",",")
keyword_string
# keyword_string = '(michael,kors%,35f6gfel3l,jet,set,travel,vanilla/acorn,large,carryall,tote,handbag)'

'(michael,kors,35f6gfel3l,jet,set,travel,vanilla/acorn,large,carryall,tote,handbag)'

### Calling the EBAY API

In [11]:
# call EBAY API and get offers, based on the keyword_string value.
from ebaysdk.finding import Connection as Finding
from ebaysdk.exception import ConnectionError

try:
    api = Finding(siteid='EBAY-US', appid="MethodDa-MethodDa-PRD-babd43d69-a4097943", config_file=None)
    response = api.execute('findItemsByKeywords', {'keywords': keyword_string})
    # print(response.dict())
except ConnectionError as e:
    print(e)
    print(e.response.dict())

In [12]:
with open('image_G_output.json', 'w') as outfile:
    json.dump(response.dict(), outfile)

In [13]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content,'lxml')

### Creating Pandas DataFrame from the image classification output, containing Item ID's, Title, URL, Price

In [14]:
# create empty DataFrame
col_names = ('Item_ID','Item_Title','Item_URL','Item_Price')
deals_df = pd.DataFrame(columns = col_names)
#
totalentries = int(soup.find('totalentries').text)
items = soup.find_all('item')
item_id = soup.find_all('itemid')
item_title = soup.find_all('title')
item_url = soup.find_all('viewitemurl')
item_price = soup.find_all('currentprice')
# add in all deals found, into the pandas dataframe
for rows in range(0,len(items)):
    deals_df = deals_df.append([{'Item_ID':item_id[rows].text,
                                 'Item_Title':item_title[rows].text,
                                 'Item_URL':item_url[rows].text,
                                 'Item_Price':item_price[rows].text}], 
                               ignore_index=True)
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

deals_df.style.format({'Item_URL': make_clickable})

,Item_ID,Item_Title,Item_URL,Item_Price
0,182974871681,Women Lady Leather Backpack Handbag Girls Shoulder Travel School Bag Rucksack,http://www.ebay.com/itm/Women-Lady-Leather-Backpack-Handbag-Girls-Shoulder-Travel-School-Bag-Rucksack-/182974871681,14.98
1,253485036473,Women Ladies Leather Shoulder Bag Tote Purse Handbag Messenger Crossbody Satchel,http://www.ebay.com/itm/Women-Ladies-Leather-Shoulder-Bag-Tote-Purse-Handbag-Messenger-Crossbody-Satchel-/253485036473?var=552668795461,7.99
2,302376737752,New Women Bags Purse Shoulder Handbag Tote Messenger Hobo Satchel Bag Cross Body,http://www.ebay.com/itm/New-Women-Bags-Purse-Shoulder-Handbag-Tote-Messenger-Hobo-Satchel-Bag-Cross-Body-/302376737752?var=601210178912,8.99
3,302376737752,New Women Bags Purse Shoulder Handbag Tote Messenger Hobo Satchel Bag Cross Body,http://www.ebay.com/itm/New-Women-Bags-Purse-Shoulder-Handbag-Tote-Messenger-Hobo-Satchel-Bag-Cross-Body-/302376737752?var=601210178910,8.99
4,302376737752,New Women Bags Purse Shoulder Handbag Tote Messenger Hobo Satchel Bag Cross Body,http://www.ebay.com/itm/New-Women-Bags-Purse-Shoulder-Handbag-Tote-Messenger-Hobo-Satchel-Bag-Cross-Body-/302376737752?var=601210178911,8.99
5,302376737752,New Women Bags Purse Shoulder Handbag Tote Messenger Hobo Satchel Bag Cross Body,http://www.ebay.com/itm/New-Women-Bags-Purse-Shoulder-Handbag-Tote-Messenger-Hobo-Satchel-Bag-Cross-Body-/302376737752?var=601210178913,8.99
6,302376737752,New Women Bags Purse Shoulder Handbag Tote Messenger Hobo Satchel Bag Cross Body,http://www.ebay.com/itm/New-Women-Bags-Purse-Shoulder-Handbag-Tote-Messenger-Hobo-Satchel-Bag-Cross-Body-/302376737752?var=601210178914,8.99
7,253485036473,Women Ladies Leather Shoulder Bag Tote Purse Handbag Messenger Crossbody Satchel,http://www.ebay.com/itm/Women-Ladies-Leather-Shoulder-Bag-Tote-Purse-Handbag-Messenger-Crossbody-Satchel-/253485036473?var=552668795459,9.59
8,311379928392,Women's Genuine Leather Handbag Cross Body Bag Shoulder Bag Organizer Mini Purse,http://www.ebay.com/itm/Womens-Genuine-Leather-Handbag-Cross-Body-Bag-Shoulder-Bag-Organizer-Mini-Purse-/311379928392?var=610466580570,11.99
9,253485036473,Women Ladies Leather Shoulder Bag Tote Purse Handbag Messenger Crossbody Satchel,http://www.ebay.com/itm/Women-Ladies-Leather-Shoulder-Bag-Tote-Purse-Handbag-Messenger-Crossbody-Satchel-/253485036473?var=552668795462,9.59


In [15]:
HEADER = '''
<html>
    <head>
        <style>
            .df tbody tr:last-child { background-color: #FF0000; }
        </style>
    </head>
    <body>
'''
FOOTER = '''
    </body>
</html>
'''
with open('Ebay_deals.html', 'w') as f:
    f.write(HEADER)
    f.write(deals_df.to_html(classes='deals_df'))
    f.write(FOOTER)